In [63]:
from unicodedata import category
import torch
from tqdm.notebook import tqdm
import numpy as np 
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import f1_score
import random
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import json

In [81]:
#getting unique categores this will be the first function
def pretraining(df,companyname,instanceid):
    possible_labels = df.category.unique()
    label_dict = {}
    for index, possible_label in enumerate(possible_labels):
        label_dict[possible_label] = index
    category_dict = {}
    count = 0
    for category in label_dict.keys():
        category_dict[count] = category
        count = count + 1
    category_file_name = "category" + companyname + str(instanceid) + ".txt"
    text_file = open(category_file_name, "w")
    categories = json.dumps(category_dict)
    text_file.write(categories)
    text_file.close()
    return category_dict


def training(df,companyname,instanceid):
    possible_labels = df.category.unique()
    label_dict = {}
    for index, possible_label in enumerate(possible_labels):
        label_dict[possible_label] = index
    df.category = df['category'].map(label_dict)
    X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                    df.category.values, 
                                                    test_size=0.15, 
                                                    random_state=42,
                                                    stratify=df.category.values)
    df['data_type'] = ['not_set']*df.shape[0]
    df.loc[X_train, 'data_type'] = 'train'
    df.loc[X_val, 'data_type'] = 'val'
    df.groupby(['category', 'data_type']).count()

    tokenizer = BertTokenizer.from_pretrained(
        'bert-base-uncased',
        do_lower_case=True
    )
    print(df)
    print("df[df.data_type=='train'].question.values",type(df[df.data_type=='train'].question.values))
    print(df[df.data_type=='train'].question.values)
    
    encoded_data_train = tokenizer.batch_encode_plus(
        
        df[df.data_type=='train'].question.values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=256,
        return_tensors='pt'
    )
    
    encoded_data_val = tokenizer.batch_encode_plus(
        df[df.data_type=='val'].question.values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=256,
        return_tensors='pt'
    )
    input_ids_train = encoded_data_train['input_ids']
    attention_masks_train = encoded_data_train['attention_mask']
    labels_train = torch.tensor(df[df.data_type=='train'].category.values)

    input_ids_val = encoded_data_val['input_ids']
    attention_masks_val = encoded_data_val['attention_mask']
    labels_val = torch.tensor(df[df.data_type=='val'].category.values)

    dataset_train = TensorDataset(input_ids_train, 
                                attention_masks_train,
                                labels_train)

    dataset_val = TensorDataset(input_ids_val, 
                                attention_masks_val,
                            labels_val)
    model = BertForSequenceClassification.from_pretrained(
                                        'bert-base-uncased', 
                                        num_labels = len(label_dict),
                                        output_attentions = False,
                                        output_hidden_states = False
                                        )
    batch_size = 4

    dataloader_train = DataLoader(
        dataset_train,
        sampler=RandomSampler(dataset_train),
        batch_size=batch_size
    )

    dataloader_val = DataLoader(
        dataset_val,
        sampler=RandomSampler(dataset_val),
        batch_size=32
    )
    optimizer = AdamW(
        model.parameters(),
        lr = 1e-5,
        eps = 1e-8
    )
    epochs = 1

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps = len(dataloader_train)*epochs
    )
    def f1_score_func(preds, labels):
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        return f1_score(labels_flat, preds_flat, average = 'weighted')

    def accuracy_per_class(preds, labels):
        label_dict_inverse = {v: k for k, v in label_dict.items()}
        
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        
        for label in np.unique(labels_flat):
            y_preds = preds_flat[labels_flat==label]
            y_true = labels_flat[labels_flat==label]
            print(f'Class: {label_dict_inverse[label]}')
            print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')
            

    seed_val = 17
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    def evaluate(dataloader_val):

        model.eval()
        
        loss_val_total = 0
        predictions, true_vals = [], []
        
        for batch in tqdm(dataloader_val):
            
            batch = tuple(b.to(device) for b in batch)
            
            inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'labels':         batch[2],
                    }

            with torch.no_grad():        
                outputs = model(**inputs)
                
            loss = outputs[0]
            logits = outputs[1]
            loss_val_total += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = inputs['labels'].cpu().numpy()
            predictions.append(logits)
            true_vals.append(label_ids)
        
        loss_val_avg = loss_val_total/len(dataloader_val) 
        
        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)
                
        return loss_val_avg, predictions, true_vals
    for epoch in tqdm(range(1, epochs+1)):
        model.train()
        loss_train_total = 0
        
        progress_bar = tqdm(dataloader_train, 
                            desc='Epoch {:1d}'.format(epoch), 
                            leave=False, 
                            disable=False)
        
        for batch in progress_bar:
            model.zero_grad()
            batch = tuple(b.to(device) for b in batch)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1],
                'labels': batch[2]
            }
            
            outputs = model(**inputs)
            loss = outputs[0]
            loss_train_total +=loss.item()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
        
        torch.save(model.state_dict(), f'BERT_ft_Epoch{epoch}.model')
        
        tqdm.write('\nEpoch {epoch}')
        
        loss_train_avg = loss_train_total/len(dataloader_train)
        tqdm.write(f'Training loss: {loss_train_avg}')
        
        val_loss, predictions, true_vals = evaluate(dataloader_val)
        val_f1 = f1_score_func(predictions, true_vals)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (weighted): {val_f1}')
    Bert_model_Name = "BertModel" + companyname + str(instanceid) 
    torch.save(model,Bert_model_Name)
    



,question,category
0,gm,Greeting
1,what is data science?,course
2,i want talk with agent,Greetings
3,bye,Greetings
4,how do i reset my password?,FAQ
...,...,...
268,want to know about emi payment options,payment
269,contact sales,course
270,english books,#
271,semi english books,#


In [83]:
dct = {}
for i in range(len(data)):
    if data.iloc[i]['category'] in dct.keys():
        dct[data.iloc[i]['category']] = dct[data.iloc[i]['category']] + 1
    else:
        dct[data.iloc[i]['category']] = 0
for key in dct:
    if dct[key] < 2:
        index_names = data[data['category'] == key].index
        data.drop(index_names, inplace = True)


In [84]:
data

,question,category
0,gm,Greeting
1,what is data science?,course
2,i want talk with agent,Greetings
3,bye,Greetings
4,how do i reset my password?,FAQ
...,...,...
267,hi,greeting
268,want to know about emi payment options,payment
269,contact sales,course
270,english books,#


In [85]:
data.index = range(len(data))

In [86]:
data

,question,category
0,gm,Greeting
1,what is data science?,course
2,i want talk with agent,Greetings
3,bye,Greetings
4,how do i reset my password?,FAQ
...,...,...
263,hi,greeting
264,want to know about emi payment options,payment
265,contact sales,course
266,english books,#


In [87]:
dct = {}
dct['question'] = data['question'].values
dct['category'] = data['category'].values
datanew = pd.DataFrame(dct)
datanew = datanew.dropna()



In [88]:
datanew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263 entries, 0 to 267
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  263 non-null    object
 1   category  263 non-null    object
dtypes: object(2)
memory usage: 6.2+ KB


In [89]:
companyname = "Amazon"
instanceid = 2
categories = pretraining(datanew,companyname,instanceid) 
training = training(datanew,companyname,instanceid)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


                                   question  category data_type
0                                        gm         0     train
1                     what is data science?         1     train
2                    i want talk with agent         2     train
3                                       bye         2     train
4               how do i reset my password?         3     train
..                                      ...       ...       ...
263                                      hi         7     train
264  want to know about emi payment options         9       val
265                           contact sales         1     train
266                           english books        10     train
267                      semi english books        10     train

[263 rows x 3 columns]
df[df.data_type=='train'].question.values <class 'numpy.ndarray'>
['gm' 'what is data science?' 'i want talk with agent' 'bye'
 'how do i reset my password?'
 'how to configure asktalos bot into website?'
 'h

C:\Users\thewa\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenc

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/56 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 2.093557683484895


  0%|          | 0/2 [00:00<?, ?it/s]

Validation loss: 1.7978473901748657
F1 Score (weighted): 0.28421052631578947
